<a href="https://colab.research.google.com/github/shahriyar-zaman/Pytorch_Learning/blob/main/Knowledge_Distilation_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To create a knowledge distillation trainer
alpha = Control the relative weightt of the distillation loss

T = how much the probability distribution of the labels should be smoothed

In [ ]:
!pip install transformers

In [ ]:
from transformers import TrainingArguments

In [ ]:
class KnowledgeDistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha = 0.5, temperature = 2.0, **kwargs):
    super().__init__(*args, **kwargs)
    self.alpha = alpha
    self.temperature = temperature

2. Code for new Loss function / KD Loss & Final Loss function.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

In [ ]:
class KnowledgeDistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model = None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher_model = teacher_model

  def compute_loss(self, model, inputs, return_outputs = False):
    #Extract cross entropy loss and logits from studnet model
    outputs_student  = model(**inputs)
    logss_ce = output_student.loss
    logits_student = outputs_student.logits

    #extract logits from teacher
    outputs_teacher = self.teacher_model.logits

    #Compute Distillation Loss by Softening Probabilities
    loss_fct = nn.KLDivLoss(reduction = "batchmean")
    #we avearge the losses over the  batch dimension

    loss_kd = self.args.temperature**2 * loss_fct(
        F.log_sofmax(logits_student/self.args.temperature, dim = 1),
        F.softmax(logits_teacher/self.args.temperature, dim = 1)
    )
    #Weighted student loss
    loss = self.args.alpha * loss_ce + (1.0 - self.args.alpha) * loss_kd
    return (loss, outputs_student) if return_outputs else loss


# How to choose a good student model

Select DistilBert as our student model



#Load Dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
clinc = load_dataset("clinc_oos", "plus")
#plus config - out of scope training example

In [ ]:
sample = clinc['train'][0]
print(sample)

{'text': 'what expression would i use to say i love you if i were an italian', 'intent': 61}


In [ ]:
intents = clinc['train'].features['intent']
intent = intents.int2str(sample['intent'])
print(intent)

translate


# Preprocess or Tokenize the Dataset

In [ ]:
from transformers import AutoTokenizer

In [ ]:
student_checkpoint = "distilbert-base-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_checkpoint)

In [ ]:
def tokenize_text(batch):
  return student_tokenizer(batch['text'], truncation = True)

In [ ]:
clinc_tokenized = clinc.map(tokenize_text, batched = True, remove_columns = ['text'])
clinc_tokenized = clinc_tokenized.rename_column("intent", "labels")

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

#Defining Distillation Trainer

In [ ]:
import numpy as np
from datasets import load_metric

In [ ]:
accuracy = load_metric("accuracy")

<ipython-input-16-ef74769317a9>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy = load_metric("accuracy")


In [ ]:
from sklearn.metrics import accuracy_score
def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis =  1)
  return accuracy_score.compute(predictions = predictions, references = labels)

# Define Training Argumnets for Distillation Trainer

In [ ]:
batch_size = 48
finetuned_student_checkpoint = "distilbert-base-uncased-finetunrf-clinc-dataset-student"

In [ ]:
student_training_args = KnowledgeDistillationTrainingArguments(
    output_dir = finetuned_student_checkpoint,
    evaluation_strategy = "epoch",
    num_train_epochs = 5,
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    alpha = 1,
    weight_decay = 0.01
)

FrozenInstanceError: ignored

In [ ]:
!pip install accelerate>=0.20.1